# ML Training with Alluxio

In [ ]:
!pip3 install -q torch_tb_profiler

In [ ]:
import time
import warnings

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)

### Checking the device used for training

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device}")

## Section 1: Data Preparation

Here, we configure necessary (hyper-)parameters and create the PyTorch data loader.

We use FUSE to mount the Alluxio dataset into the training node. So users can conviniently load 
datasets from Alluxio as if they are on the local disk.

In [ ]:
batch_size = 128
num_workers = 16
dataset_path = "../data/imagenet-mini/train"
output_model_path = "../ai-demo.pth"
learning_rate = 0.001
num_epochs = 3
profiler_enabled = True
profiler_log_path = "../log/ai-demo"

In [ ]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=45),
        transforms.ColorJitter(
            brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5
        ),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
        ),
    ]
)

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(
    root=dataset_path, transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
)

## Section 2: Setup the Model

We fine-tune the ResNet18 model against a subset of the ImageNet dataset.

In [ ]:
model = resnet18(weights=ResNet18_Weights.DEFAULT)
model = torch.nn.DataParallel(model)
model = model.to(device)

## Section 3: Setup the PyTorch Profiler

We setup the PyTorch profiler with the TensorBoard for the visualization of GPU utilization and other metrics.

In [ ]:
profiler = None
if profiler_enabled:
    profiler = torch.profiler.profile(
        schedule=torch.profiler.schedule(
            wait=0, warmup=0, active=1, repeat=1
        ),
        on_trace_ready=torch.profiler.tensorboard_trace_handler(
            profiler_log_path
        ),
    )
    profiler.start()

## Section 4: Model Training

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate
)

start_time = time.perf_counter()
print(f"Started training at the timestamp {time.perf_counter()}")

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # Move input and label tensors to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero out the optimization
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

    print(
        f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():0.4f} at the timestamp {time.perf_counter()}"
    )

    if profiler_enabled:
        profiler.step()

print(f"Finished Training, Loss: {loss.item():0.4f}")

end_time = time.perf_counter()
print(f"Training time in {end_time - start_time:0.4f} seconds")

if profiler_enabled:
    profiler.stop()
    print("The profiler is completed. Please open the TensorBoard to browse the metrics.")

## Save the Trained Model

In [ ]:
torch.save(model.state_dict(), output_model_path)
print(f"Saved PyTorch AI demo model to {output_model_path}")